### tensorflow로 Face Detection 수행하기
* WiderFace 데이터세트로 Pretrained된 Tensorflow graph 모델을 다운로드 받아 이를 이용해 Face Detection 수행. 

### 본 실습 예제는 GPU를 활용하므로 상단 메뉴에서 런타임->런타임 유형 변경에서 GPU를 선택해 주십시요. 


#### tensorflow, keras 설치 및 강의 실습코드/데이터 Download

**공지**

현재(2020년 8월 11일) Colab에서 GPU 커널 적용시 tensorflow 1.13으로 downgrade가 되지 않습니다. 
때문에 colab에서는 Segmentation 학습 시 tensorflow 1.15, keras 2.3 을 설치하겠습니다. 

Colab 버전colab에서 pip 명령어를 이용하여 tensorflow 1.15, keras 2.3를 소스코드 커널 기동시 마다 설치해야 합니다

Colab의 tensorflow는 2020년 기준으로 2.2 이며, keras는 2.3입니다. 실습코드는 tensorflow 1.13과 1.15, keras 2.2와 2.3 기준으로 되어 있으므로 이를 downgrade해야 합니다.

pip를 이용하여 tensorflow 1.15을 설치하면 자동으로 downgrade 됩니다.

OpenCV는 Colab에서 이미 설치 되어 있으니 추가설치는 필요 없습니다.

강의 실습코드와 데이터는 https://github.com/chulminkw/DLCV.git 에서 다운로드 할 수 있습니다. 

In [ ]:
# 현재 디렉토리는 /content이며 이 디렉토리를 기준으로 실습코드와 데이터를 다운로드 합니다. 
!pwd
!rm -rf DLCV
!git clone https://github.com/chulminkw/DLCV.git
# DLCV 디렉토리가 Download되고 DLCV 밑에 Detection과 Segmentation 디렉토리가 있는 것을 확인
!ls -lia 
!ls -lia DLCV

In [ ]:
#현재(2020년 8월 11일) Colab에서 GPU 커널 적용시 tensorflow 1.13으로 downgrade가 되지 않습니다. 
#때문에 colab에서는 Segmentation 학습 시 tensorflow 1.15, keras 2.3 을 설치하겠습니다.
# tensorflow 1.15을 설치합니다. 자동으로 tensorflow 2.2가 1.15로 downgrade 됩니다. 
!pip install tensorflow-gpu==1.15.2 
# keras 2.3를 설치합니다. 
!pip install keras==2.3.0

In [ ]:
# tensorflow 1.15과 keras 2.3 version 확인.  
import tensorflow as tf
import keras

print(tf.__version__)
print(keras.__version__)

#### Tensorflow Face Detector
* Github : https://github.com/yeephycho/tensorflow-face-detection
* SSD + MobileNet기반으로 Pretrained된 모델 다운로드 
* https://github.com/yeephycho/tensorflow-face-detection/raw/master/model/frozen_inference_graph_face.pb

In [ ]:
### 코랩 버전은 아래를 이용하여 다시 pretrained 모델을 다운로드 받아야 합니다.

In [ ]:
!rm -rf /content/DLCV/Detection/ssd//pretrained
!mkdir /content/DLCV/Detection/ssd/pretrained
# pretrained 디렉토리가 생성되었는지 확인 합니다. 
%cd /content/DLCV/Detection/ssd
!ls

### coco 데이터 세트로 pretrained 된 ssd 파일과 config용 graph pbtxt 다운로드 
%cd /content/DLCV/Detection/ssd/pretrained
!echo "##### downloading pretrained ssd face weight file"
!wget https://github.com/yeephycho/tensorflow-face-detection/raw/master/model/frozen_inference_graph_face.pb

!echo "##### check out pretrained ssd face weight file"
!ls /content/DLCV/Detection/ssd/pretrained

#### SSD + Mobilenet Pretrained된 모델 로딩하여 Face  Detection

In [ ]:
import numpy as np
import tensorflow as tf
import cv2
import time
import matplotlib.pyplot as plt
import os
%matplotlib inline

#코랩 버전은 default_dir과 default_rcnn_dir 절대 경로를 이용합니다. os.path.join()으로 상세 파일/디렉토리를 지정합니다. 
default_dir = '/content/DLCV'
default_ssd_dir='/content/DLCV/Detection/ssd'

#inference graph를 읽음. .
with tf.gfile.FastGFile(os.path.join(default_ssd_dir, 'pretrained/frozen_inference_graph_face.pb'), 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    
with tf.Session() as sess:
    # Session 시작하고 inference graph 모델 로딩 
    sess.graph.as_default()
    tf.import_graph_def(graph_def, name='')
    
    # 입력 이미지 생성 및 BGR을 RGB로 변경 
    img = cv2.imread(os.path.join(default_dir, 'data/image/EPL01.jpg'))
    draw_img = img.copy()
    rows = img.shape[0]
    cols = img.shape[1]
    inp = cv2.resize(img, (300, 300))
    # OpenCV로 입력받은 BGR 이미지를 RGB 이미지로 변환 
    inp = inp[:, :, [2, 1, 0]] 

    start = time.time()
    # Object Detection 수행. 
    out = sess.run([sess.graph.get_tensor_by_name('num_detections:0'),
                    sess.graph.get_tensor_by_name('detection_scores:0'),
                    sess.graph.get_tensor_by_name('detection_boxes:0'),
                    sess.graph.get_tensor_by_name('detection_classes:0')],
                   feed_dict={'image_tensor:0': inp.reshape(1, inp.shape[0], inp.shape[1], 3)})
    
    green_color=(0, 255, 0)
    red_color=(0, 0, 255)
    
    # Bounding Box 시각화 
    # Detect된 Object 별로 bounding box 시각화 
    num_detections = int(out[0][0])
    for i in range(num_detections):
        # class id와 object class score, bounding box정보를 추출
        classId = int(out[3][0][i])
        score = float(out[1][0][i])
        bbox = [float(v) for v in out[2][0][i]]
        if score > 0.4:
            left = bbox[1] * cols
            top = bbox[0] * rows
            right = bbox[3] * cols
            bottom = bbox[2] * rows
            # cv2의 rectangle(), putText()로 bounding box의 클래스명 시각화 
            cv2.rectangle(draw_img, (int(left), int(top)), (int(right), int(bottom)), green_color, thickness=2)
            caption = "face: {:.4f}".format(score)
            print(caption)
            cv2.putText(draw_img, caption, (int(left), int(top - 5)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, red_color, 2)
    
    print('Detection 수행시간:',round(time.time() - start, 2),"초")
    
img_rgb = cv2.cvtColor(draw_img, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(12, 12))
plt.imshow(img_rgb)
        

In [ ]:
def get_tensor_detected_image(sess, img_array, use_copied_array):
    
    rows = img_array.shape[0]
    cols = img_array.shape[1]
    if use_copied_array:
        draw_img = img_array.copy()
    else:
        draw_img = img_array
    
    inp = cv2.resize(img_array, (300, 300))
    inp = inp[:, :, [2, 1, 0]]  # BGR2RGB
    
    start = time.time()
    # Object Detection 수행. 
    out = sess.run([sess.graph.get_tensor_by_name('num_detections:0'),
                    sess.graph.get_tensor_by_name('detection_scores:0'),
                    sess.graph.get_tensor_by_name('detection_boxes:0'),
                    sess.graph.get_tensor_by_name('detection_classes:0')],
                   feed_dict={'image_tensor:0': inp.reshape(1, inp.shape[0], inp.shape[1], 3)})
    
    green_color=(0, 255, 0)
    red_color=(0, 0, 255)
    
    # Bounding Box 시각화 
    # Detect된 Object 별로 bounding box 시각화 
    num_detections = int(out[0][0])
    for i in range(num_detections):
        # class id와 object class score, bounding box정보를 추출
        classId = int(out[3][0][i])
        score = float(out[1][0][i])
        bbox = [float(v) for v in out[2][0][i]]
        if score > 0.4:
            left = bbox[1] * cols
            top = bbox[0] * rows
            right = bbox[3] * cols
            bottom = bbox[2] * rows
            # cv2의 rectangle(), putText()로 bounding box의 클래스명 시각화 
            cv2.rectangle(draw_img, (int(left), int(top)), (int(right), int(bottom)), green_color, thickness=2)
            caption = "face: {:.4f}".format(score)
            cv2.putText(draw_img, caption, (int(left), int(top - 5)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, red_color, 2)
    
    print('Detection 수행시간:',round(time.time() - start, 3),"초")
    return draw_img

#코랩 버전은 default_dir과 default_rcnn_dir 절대 경로를 이용합니다. os.path.join()으로 상세 파일/디렉토리를 지정합니다. 
default_dir = '/content/DLCV'
default_ssd_dir='/content/DLCV/Detection/ssd'

#inference graph를 읽음. .
with tf.gfile.FastGFile(os.path.join(default_ssd_dir, 'pretrained/frozen_inference_graph_face.pb'), 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    
with tf.Session() as sess:
    # Session 시작하고 inference graph 모델 로딩 
    sess.graph.as_default()
    tf.import_graph_def(graph_def, name='')
    
    # 입력 이미지 생성, Object Detection된 image 반환, 반환된 image의 BGR을 RGB로 변경 
    img = cv2.imread(os.path.join(default_dir, 'data/image/EPL01.jpg'))
    draw_img = get_tensor_detected_image(sess, img, True)

img_rgb = cv2.cvtColor(draw_img, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(12, 12))
plt.imshow(img_rgb)

#### tensorflow로 SSD+ Inception 기반 video Object Detection 수행

In [ ]:
# 코랩 버전은 아래 코드를 이용합니다.
from IPython.display import HTML
from base64 import b64encode
mp4 = open('/content/DLCV/data/video/InfiniteWar01.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=800 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [ ]:
default_dir = '/content/DLCV'
default_ssd_dir='/content/DLCV/Detection/ssd'


video_input_path = os.path.join(default_dir, 'data/video/InfiniteWar01.mp4')
# linux에서 video output의 확장자는 반드시 avi 로 설정 필요. 
video_output_path = os.path.join(default_dir, 'data/output/InfiniteWar01_ssd.avi')

cap = cv2.VideoCapture(video_input_path)

codec = cv2.VideoWriter_fourcc(*'XVID')

vid_size = (round(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
vid_fps = cap.get(cv2.CAP_PROP_FPS)
    
vid_writer = cv2.VideoWriter(video_output_path, codec, vid_fps, vid_size) 

frame_cnt = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print('총 Frame 갯수:', frame_cnt, 'FPS:', vid_fps )

green_color=(0, 255, 0)
red_color=(0, 0, 255)

# SSD+Inception inference graph를 읽음. .
with tf.gfile.FastGFile(os.path.join(default_ssd_dir, 'pretrained/frozen_inference_graph_face.pb'), 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    
with tf.Session() as sess:
    # Session 시작하고 inference graph 모델 로딩 
    sess.graph.as_default()
    tf.import_graph_def(graph_def, name='')
    index = 0
    while True:
        hasFrame, img_frame = cap.read()
        if not hasFrame:
            print('더 이상 처리할 frame이 없습니다.')
            break

        draw_img_frame = get_tensor_detected_image(sess, img_frame, False)
        vid_writer.write(draw_img_frame)
    # end of while loop

vid_writer.release()
cap.release()  


In [ ]:
## colab 버전은 Object Detection 적용된 영상 파일을 google drive에서 download 해야 합니다. 이를 위해 google drive를 colab에 mount 수행. 
import os, sys 
from google.colab import drive 

drive.mount('/content/gdrive')

In [ ]:
## colab 버전은 Object Detection 적용된 영상 파일을 google drive에서 download 해야 합니다. 
## My Drive 디렉토리 이름에 공란이 있으므로 ' '로 묶습니다. 
!cp /content/DLCV/data/output/InfiniteWar01_ssd.avi '/content/gdrive/My Drive/InfiniteWar01_ssd.avi'
